In [96]:
import pandas as pd
import numpy as np
import random 
import os
from IPython.display import clear_output

In [124]:
df = pd.read_excel("単語データ.xlsx",header=None)
df.columns = ("number","English","Japanese")
df = df.set_index("number")
wrong = df["Japanese"].unique()
wrong_list = pd.DataFrame(wrong)
df.head()

,English,Japanese
number,,
1,be free from[of] ~,~がない
2,be free to do,自由に~する
3,on account of ~,~のために
4,be bound for ~,~行きである
5,be bound to do,きっと~する


In [125]:
df.describe()

,English,Japanese
count,100,100
unique,100,81
top,be free from[of] ~,~にとって不可欠である
freq,1,2


In [351]:
df["in_flag"] = 0

display(df.head(10))
print(len(df))
for i in range(len(a.number.unique())):
    temp = a.number.unique()[i]
    if df.index.isin([a.number.unique()[i]]).any():
        df.iloc[temp-1,2] = 1

df = df.loc[df["in_flag"] == 0]


,English,Japanese,in_flag
number,,,
1,be free from[of] ~,~がない,0
2,be free to do,自由に~する,0
4,be bound for ~,~行きである,0
5,be bound to do,きっと~する,0
6,as to ~,~に関して(は),0
8,be anxious about ~,~を心配している,0
9,be anxious to do,~したがる,0
10,on and off,断続的に,0
11,for all ~,~にもかかわらず,0


92


In [373]:
def testmake(n,start,stop) -> pd.DataFrame:
    #単語テストのデータの読み込み
    full_test = pd.read_excel("単語データ.xlsx",header=None)
    full_test.columns = ("number","English","Japanese")
    full_test = full_test.set_index("number")
    #正誤データの読み込み
    recent_c = pd.read_csv("recent_correct.csv")

    #間違いの選択肢を作る
    wrong = full_test["Japanese"].unique()
    wrong_list = pd.DataFrame(wrong)

    #今回のテスト範囲を作る。最近正解したものがあれば削除する
    test_range = full_test.iloc[start-1:stop].copy()
    test_range["in_flag"] = 0
    for i in range(len(recent_c["number"].unique())):
        temp = recent_c["number"].unique()[i]
        if test_range.index.isin([temp]).any():
            test_range.iloc[temp -1,2] = 1
    
    test_range = test_range.loc[test_range["in_flag"] == 0]
    del test_range["in_flag"]

    #正解したものを除いたテスト範囲から、指定された問題数抽出する
    test_sampled = test_range.sample(n = n,replace = False)
    

    #テストの選択肢の作成
    for i in range(4):
        test_sampled["test_{}".format(i+1)] = None
    test_sampled["answer"] = None
    test = test_sampled.iloc[0:0]
  
    #選択肢に正解と問題を入れる    
    for i in range(0,len(test_sampled)):
        test_question = test_sampled.iloc[[i],:].copy()
        answer = random.randint(1,4)
        test_question["test_{}".format(answer)] = test_question["Japanese"]

        test_wrong = wrong_list.sample(n = 4)
        while pd.concat([test_wrong,test_question["Japanese"]]).nunique().sum() <= 4:
            test_wrong = wrong_list.sample(n = 4,replace=False)
        
        for wro in range(1,5):
            if test_question["test_{}".format(wro)].iloc[[0]].isnull().bool() == True:
                test_question["test_{}".format(wro)].iloc[[0]] = test_wrong.iloc[wro-1]
                
        test_question["answer"] = answer
        
        test = pd.concat([test,test_question])
        
    del test["Japanese"]
    return test

def test_play(test_df):
    cor = 0
    test_df["is_correct"] = None
    for i in range(len(test_df)):
        print(i + 1,"問目","-------------------")
        finish = True
        while finish == True:
            display(test_df.iloc[i,0:5])
            answer = int(input("答えを入力してください。もし表示されなかったら0"))
            if answer == test_df.iloc[i,5]:
                test_df.iloc[i,6] = 1
                cor += 1
                finish = False
            elif answer == 0:
                finish = True
            else:
                test_df.iloc[i,6] = 0
                finish = False
        
        if (i + 1) % 5 == 0 and i != 0:
            clear_output()
        
        



In [377]:
test_test = testmake(n = 5,start = 1,stop = 40)
test_test

,English,test_1,test_2,test_3,test_4,answer
number,,,,,,
33,stick to ~,~に飽きる;~にうんざりしている,さしあたり;当分の間(は),~にくっつく;(主義・決定など)を堅持する,(~に)屈服する,3
30,for one's (own) part,1(寝ないで)起きている 2(寝た状態から)上半身を起こす 3きちんと座る,(~に)屈する;~に道を譲る,~としては;~に関する限りは,~を心配している,3
34,be true of ~,~に当てはまる,それと反対の[に],(スイッチなど)を消す;(水・ガスなど)を止める;(明かりなどが)消える,現在は;目下,1
16,keep back ~,~を最大限に利用する,断続的に,要するに;つまり,~を制止する;(真相など)を隠す,4
38,specialize in ~,(人)によく知られている,AをBと間違える,~を専攻する;~を専門にする,~を見失う,3


In [378]:
test_play(test_df=test_test)

In [379]:
test_test

,English,test_1,test_2,test_3,test_4,answer,is_correct
number,,,,,,,
33,stick to ~,~に飽きる;~にうんざりしている,さしあたり;当分の間(は),~にくっつく;(主義・決定など)を堅持する,(~に)屈服する,3,1
30,for one's (own) part,1(寝ないで)起きている 2(寝た状態から)上半身を起こす 3きちんと座る,(~に)屈する;~に道を譲る,~としては;~に関する限りは,~を心配している,3,0
34,be true of ~,~に当てはまる,それと反対の[に],(スイッチなど)を消す;(水・ガスなど)を止める;(明かりなどが)消える,現在は;目下,1,1
16,keep back ~,~を最大限に利用する,断続的に,要するに;つまり,~を制止する;(真相など)を隠す,4,1
38,specialize in ~,(人)によく知られている,AをBと間違える,~を専攻する;~を専門にする,~を見失う,3,1


In [159]:
correct_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 1].index)
miss_list = pd.DataFrame(test_test.loc[test_test["is_correct"] == 0].index)

In [160]:
display(correct_list)
display(miss_list)

,number
0,23
1,45
2,39
3,27
4,18


,number
0,21
1,41
2,50
3,8
4,31
5,29


In [382]:
df = pd.read_csv("recent_correct.csv")
df += 1
df

,number,time_flag
0,8,2
1,23,2
2,40,2
3,4,2
4,24,1
5,46,1
6,40,1
7,28,1
8,19,1
